Переводчики на контурных моделях

## mBART-50 (1024) Transformers (Hugging Face).
https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt

In [ ]:
#@title Установка зависимостей

!pip install transformers==4.35.0
!pip install sentencepiece==0.1.99
!pip install sacremoses==0.1.1
!pip install accelerate==0.24.1

In [ ]:
!git lfs install
!git clone https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt

Git LFS initialized.
Cloning into 'mbart-large-50-many-to-many-mmt'...
remote: Enumerating objects: 43, done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (43/43), 7.57 KiB | 516.00 KiB/s, done.
Filtering content: 100% (6/6), 11.38 GiB | 44.87 MiB/s, done.


In [ ]:
#@title Использование конвейера (pipeline)
from transformers import pipeline

# "facebook/mbart-large-50-many-to-many-mmt"
pipe = pipeline("translation",
                model="/content/mbart-large-50-many-to-many-mmt",
                src_lang="ru_RU",
                tgt_lang="en_XX",
                max_length=1024,
                device='cuda')
text = '...'
pipe(text)

In [ ]:
#@title Токены
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


def tokens_mBART50(text, lang="en_XX"):
    tokenizer = MBart50TokenizerFast.from_pretrained("/content/mbart-large-50-many-to-many-mmt")
    tokenizer.src_lang = lang
    token_list = tokenizer.encode(text)
    # К-во токенов, список токенов, список частей текста
    return len(token_list), token_list, [tokenizer.decode(n) for n in token_list]


n, ln, ls = tokens_mBART50('A small team of artificial intelligence researchers at the Institute for Intelligent Computing')
print("Количество токенов во входной последовательности:", n)
print(ln)
print(ls)

Количество токенов во входной последовательности: 18
[250004, 62, 19336, 7175, 111, 66733, 6, 130687, 25188, 1314, 99, 70, 43975, 100, 235398, 159415, 214, 2]
['en_XX', 'A', 'small', 'team', 'of', 'artificial', '', 'intelligence', 'research', 'ers', 'at', 'the', 'Institute', 'for', 'Intelligent', 'Comput', 'ing', '</s>']


In [ ]:
text = """A small team of artificial intelligence researchers at the Institute for Intelligent Computing, Alibaba Group,
demonstrates, via videos they created, a new AI app that can accept a single photograph of a person's face and a soundtrack
of someone speaking or singing and use them to create an animated version of the person speaking or singing the voice track.
The group has published a paper describing their work on the arXiv preprint server.

Prior researchers have demonstrated AI applications that can process a photograph of a face and use it to create a
semi-animated version. In this new effort, the team at Alibaba has taken this a step further by adding sound. And perhaps,
just as importantly, they have done so without the use of 3D models or even facial landmarks. Instead, the team has used
diffusion modeling based on training an AI on large datasets of audio or video files. In this instance, the team used
approximately 250 hours of such data to create their app, which they call Emote Portrait Alive (EMO).

By directly converting the audio waveform into video frames, the researchers created an application that captures subtle
human facial gestures, quirks of speech and other characteristics that identify an animated image of a face as human-like.
The videos faithfully recreate the likely mouth shapes used to form words and sentences, along with expressions typically
associated with them."""


print("Количество токенов во входном тексте:", tokens_mBART50(text)[0])

Количество токенов во входном тексте: 307


In [ ]:
#@title Ручная загрузка модели
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch


device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = MBartForConditionalGeneration.from_pretrained(
                   "/content/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained(
                   "/content/mbart-large-50-many-to-many-mmt")


def tranlate_mBART50(text, tokenizer, device, model, input_lang_code="en_XX", output_lang_code="ru_RU"):
    tokenizer.src_lang = input_lang_code
    encoded_ru = tokenizer(text, return_tensors="pt").to(device)
    generated_tokens = model.generate(
                **encoded_ru,
                forced_bos_token_id=tokenizer.lang_code_to_id[output_lang_code],
                max_length=1024)
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
%%time
#@title mBART-50 - Russian, Spanish, French
lang_codes = ['ru_RU', 'es_XX', 'fr_XX']
for lang_code in lang_codes:
    print('\n', tranlate_mBART50(text, tokenizer, device, model, output_lang_code=lang_code))


 ['Небольшая команда исследователей искусственного интеллекта в Институте интеллектуального вычисления, Alibaba Group, демонстрирует, через видео, которые они создали, новую программу искусственного интеллекта, которая может принять одну фотографию лица человека и саундтрек кого-то говорить или петь и использовать их для создания анимированной версии человека говорить или петь голос трек. Группа опубликовала статью, описывающую свою работу на arXiv предварительного печати сервера. Предыдущие исследователи продемонстрировали приложения искусственного интеллекта, которые могут обрабатывать фотографию лица и использовать ее для создания полуанимированной версии. В этом новом усилии, команда в Alibaba взяла этот шаг дальше путем добавления звука. И, возможно, столь же важно, они сделали это без использования 3D моделей или даже линейных знаков. Вместо этого, команда использовала диффузионное моделирование, основанное на обучении искусственного интеллекта на больших наборах данных аудио ил

Входная последовательность на английском 307 токенов, но модель mBART-50 почему-то не перевела до конца на русский и испанский. На французский полностью до конца.

## Saiga

### https://huggingface.co/IlyaGusev
### https://huggingface.co/IlyaGusev/saiga2_7b_gguf

In [ ]:
!pip install transformers==4.35.0
!pip install sentencepiece==0.1.99
!pip install sacremoses==0.1.1
!pip install accelerate==0.24.1

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir

In [ ]:
!wget 'https://huggingface.co/IlyaGusev/saiga2_7b_gguf/resolve/main/model-q2_K.gguf'

--2024-03-02 17:27:53--  https://huggingface.co/IlyaGusev/saiga2_7b_gguf/resolve/main/model-q2_K.gguf
Resolving huggingface.co (huggingface.co)... 99.84.108.70, 99.84.108.87, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/5c/51/5c517a9e7f5c6a5b0428792a82e7a380cfeaabb4d93e66cbb82f023ed030f30d/ecdb8ac229f6cdc3f21d5a3e95dcd26f31c355b727299e96b44ab7a7dbc34fb4?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-q2_K.gguf%3B+filename%3D%22model-q2_K.gguf%22%3B&Expires=1709659673&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTY1OTY3M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy81Yy81MS81YzUxN2E5ZTdmNWM2YTViMDQyODc5MmE4MmU3YTM4MGNmZWFhYmI0ZDkzZTY2Y2JiODJmMDIzZWQwMzBmMzBkL2VjZGI4YWMyMjlmNmNkYzNmMjFkNWEzZTk1ZGNkMjZmMzFjMzU1YjcyNzI5OWU5NmI0NGFiN2E3ZGJjMzRmYjQ%7Ecm

In [ ]:
#@title Saiga. Токены
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_mistral_7b_lora")


def tokens_saiga(text, tokenizer, lang):
    tokenizer.src_lang = lang
    token_list = tokenizer.encode(text)
    # К-во токенов, список токенов, список частей текста
    return len(token_list), token_list, [tokenizer.decode(n) for n in token_list]

In [ ]:
string = 'Небольшая команда исследователей искусственного интеллекта в Институте интеллектуальных вычислений'
n, ln, ls = tokens_saiga(string, tokenizer, lang="ru_RU")
print("Количество токенов:", n)
print(ln)
print(ls)

Количество токенов: 32
[12112, 2290, 1049, 25356, 11015, 1225, 17282, 23050, 16871, 839, 11249, 28788, 7265, 3111, 8257, 12516, 1125, 16702, 649, 18170, 17356, 1078, 8257, 12516, 1125, 19456, 8067, 4254, 4627, 2348, 5333, 3962]
['Не', 'бо', 'ль', 'шая', 'коман', 'да', 'ис', 'следова', 'телей', 'и', 'ску', 'с', 'ствен', 'ного', 'ин', 'тел', 'ле', 'кта', 'в', 'Ин', 'ститу', 'те', 'ин', 'тел', 'ле', 'кту', 'аль', 'ных', 'вы', 'чи', 'сле', 'ний']


In [ ]:
string = 'A small team of artificial intelligence researchers at the Institute for Intelligent Computing'
n, ln, ls = tokens_saiga(string, tokenizer, lang="en_EN")
print("Количество токенов:", n)
print(ln)
print(ls)

Количество токенов: 16
[330, 1741, 1918, 302, 18278, 10895, 15334, 438, 272, 8624, 354, 4666, 6095, 308, 13651, 288]
['A', 'small', 'team', 'of', 'artificial', 'intelligence', 'researchers', 'at', 'the', 'Institute', 'for', 'Int', 'ellig', 'ent', 'Comput', 'ing']


In [ ]:
#@title  Перевод Saiga
from llama_cpp import Llama


SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
SYSTEM_TOKEN = 1788
USER_TOKEN = 1404
BOT_TOKEN = 9225
LINEBREAK_TOKEN = 13

ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}

def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens

def get_system_tokens(model):
    system_message = {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
    return get_message_tokens(model, **system_message)

def interact(
    model_path,
    prompt,
    n_ctx=4096,  # Context window
    max_tokens=2000,  # Max tokens to generate
    n_gpu_layers = 2000,  # Change this value based on your model and your GPU VRAM pool.
    n_batch = 512,
    top_k=30,
    top_p=0.9,
    temperature=0.2,
    repeat_penalty=1.1
):
    model = Llama(
        model_path=model_path,
        n_ctx=n_ctx,
        n_parts=1,
        max_tokens=max_tokens,
        n_gpu_layers = n_gpu_layers,
        n_batch = n_batch,
    )

    system_tokens = get_system_tokens(model)
    tokens = system_tokens
    model.eval(tokens)

    user_message = prompt
    message_tokens = get_message_tokens(model=model, role="user", content=user_message)
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens
    print('Количество токенов на вход: ', len(tokens))
    generator = model.generate(
        tokens,
        top_k=top_k,
        top_p=top_p,
        temp=temperature,
        repeat_penalty=repeat_penalty
    )
    print('\n\n')
    for token in generator:
        token_str = model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)
        if token == model.token_eos():
            break
        print(token_str, end="", flush=True)
    print('\nКоличество всего токенов: ', len(tokens))

In [ ]:
text = """A small team of artificial intelligence researchers at the Institute for Intelligent Computing, Alibaba Group,
demonstrates, via videos they created, a new AI app that can accept a single photograph of a person's face and a soundtrack
of someone speaking or singing and use them to create an animated version of the person speaking or singing the voice track.
The group has published a paper describing their work on the arXiv preprint server.

Prior researchers have demonstrated AI applications that can process a photograph of a face and use it to create a
semi-animated version. In this new effort, the team at Alibaba has taken this a step further by adding sound. And perhaps,
just as importantly, they have done so without the use of 3D models or even facial landmarks. Instead, the team has used
diffusion modeling based on training an AI on large datasets of audio or video files. In this instance, the team used
approximately 250 hours of such data to create their app, which they call Emote Portrait Alive (EMO).

By directly converting the audio waveform into video frames, the researchers created an application that captures subtle
human facial gestures, quirks of speech and other characteristics that identify an animated image of a face as human-like.
The videos faithfully recreate the likely mouth shapes used to form words and sentences, along with expressions typically
associated with them."""

prompt = f"""Переведи текст на русский язык.
Текст: {text}
Текст на русском:
"""

interact(model_path="/content/model-q2_K.gguf", prompt=prompt)

llama_model_loader: loaded meta data with 16 key-value pairs and 291 tensors from /content/model-q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32


Количество токенов на вход:  408





Llama.generate: prefix-match hit


Исследовательская группа маленького отдела искусственного интеллекта Института для компьютерной обработки, Алибаба Групп, демонстрирует, через видео, которые они создали, новую программу искусственного интеллекта, которая может принять одну фотографию лица и звуковой трек, чтобы использовать их для создания анимированной версии голоса или пения. Группа опубликовала описание своей работы на сервисе arXiv препрофил.
Предыдущие исследователи демонстрировали приложения искусственного интеллекта, которые могут обрабатывать фотографию лица и использовать ее для создания полу-анимированной версии. В этом новом усилии, группа Алибаба взяла это еще дальше, добавив звуковой трек. И, возможно, еще важнее всего, они использовали без использования 3D моделей или даже лицевых мест. Вместо этого, группа использовала моделирование рассеивания, основанное на обучении АИ на больших базах данных звука или видео. В этом случае группа использовала приблизительно 250 часов таких данных для создания своего п